In [ ]:
import pandas as pd
from dateutil.parser import parse
import numpy as np
from multiprocessing import Pool
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import statsmodels.tsa.api as tsa
import statsmodels.graphics.tsaplots as tsp
import prophet as pr
import pmdarima as pmda
import scipy as sp

sns.set()

# 1. Loading Data

## 1.1. Raw Data

In [ ]:
cits = pd.read_csv("data/CitationsTS", parse_dates=["Date"], index_col="Date")
cits.AdjCitations = cits.AdjCitations*100
cits = cits[cits.index.year > 1979]
refs = pd.read_csv("data/ReferencesTS", parse_dates=["Date"], index_col="Date")
refs = refs[refs.index.year > 1979]

In [ ]:
fields = cits.Field.unique().tolist()
fields = [ f for f in fields if f != "overall" ]
fields.insert(0, "overall")
conditions = cits.Condition.unique().tolist()

## 1.2. Isolating TimeSeries

In [ ]:
tmp = cits[["AdjCitations", "Field", "Condition"]]
ts = tmp.loc[:,[]].reset_index().drop_duplicates().set_index("Date")

for c in conditions:
    for f in fields:
        x = tmp[(tmp["Condition"] == c) & (tmp["Field"] == f)]
        ts[(c,f)] = x.AdjCitations
ts = ts.sort_index()
ts.columns = pd.MultiIndex.from_tuples(ts.columns.tolist())
years = getattr(ts.index, "year").unique().tolist()

# 2. Preliminary Visualization

## 2.1. Full Time Series

It is evident from the first two cells that Citations and References are similarly trended. It is more useful then to confine the analysis to the Adjusted Citations only.

In [ ]:
# Visualize Citations
for c in conditions:
    fig = plt.figure(figsize = (18, 6), constrained_layout = True)
    ax = fig.add_subplot(111)
    for f in fields:
        tmp = cits[(cits["Condition"] == c) & (cits["Field"] == f)]
        ax.plot(tmp.index, tmp.Citations, label = f)
    ax.legend()
    ax.set_title("Citations - "+c)
    plt.savefig("plots/rawTS/RawCitations-"+c)

In [ ]:
# Visualize References
fig = plt.figure(figsize = (18, 6), constrained_layout=True)
ax = fig.add_subplot(111)
for f in fields:
    tmp = refs[refs["Field"] == f]
    ax.plot(tmp.index, tmp.References, label = f)
ax.legend()
ax.set_title("References")
plt.savefig("plots/rawTS/RawReferences")

In [ ]:
# Visualize %Citations
for c in conditions:
    fig = plt.figure(figsize = (18, 6), constrained_layout = True)
    ax = fig.add_subplot(111)
    for f in fields:
        ax.plot(ts[c][f], label = f)
    ax.legend()
    ax.set_title("Citations % - "+c)
    plt.savefig("plots/rawTS/Citations%-"+c)

## 2.2. Separate Visualizations

### 2.2.1. Time Series

In [ ]:
# Comparative visualisation
ylims = [(0,3), (0,75), (0,75)]
for k, c in enumerate(conditions):
    fig, axs = plt.subplots(5,4, figsize = (18,12), constrained_layout = True)
    for i,ax in enumerate(axs.flatten()):
        ts[c][fields[i]].plot(
            ax = ax, title = fields[i], legend=False, 
            color = ("r" if i == 0 else "b"))
        ax.set_ylim(ylims[k])
        ax.set_xlabel("")
    fig.suptitle("Citations % - "+c, fontsize = 16)
    fig.savefig("plots/rawTS/compare/complete_"+c)

In [ ]:
# Comparative visualisation last 5 years
ylims = [(0,3), (0,75), (0,75)]
for k, c in enumerate(conditions):
    fig, axs = plt.subplots(5,4, figsize = (18,12), constrained_layout = True)
    for i,ax in enumerate(axs.flatten()):
        tmp = ts[c][fields[i]]
        tmp[tmp.index.year >2014].plot(
            ax = ax, title = fields[i], legend=False, 
            color = ("r" if i == 0 else "b"))
        ax.set_ylim(ylims[k])
        ax.set_xlabel("")
    fig.suptitle("Citations % - "+c, fontsize = 16)
    fig.savefig("plots/rawTS/compare/5years_"+c)

### 2.2.2. Seasonal Plots

In [ ]:
clrs = np.random.choice(list(mpl.colors.XKCD_COLORS.keys()), len(years), replace=False)
for c in conditions:
    fig, axs = plt.subplots(ncols = 4, nrows = 5, figsize = (18, 12), constrained_layout = True)
    for i, ax in enumerate(axs.flatten()):
        piv = pd.pivot_table(ts[c],[fields[i]], getattr(ts.index, "month"), getattr(ts.index, "year"))
        piv.plot(legend = False, color = clrs, ax = ax, title = fields[i])
        for k,y in enumerate(years):
            ax.text(piv.shape[0]+.1, piv[-1:].values[0][k], y, color = clrs[k])
        ax.set_xlabel("")
    fig.suptitle("Citations % - "+c, fontsize = 16)
    fig.savefig("plots/rawTS/compare/seasonal_"+c)

# 3. Preliminary Analysis

## 3.1. STL decomposition

In [ ]:
def plot_decompose(dec, tit, axs):
    axs[0].set_title(tit)
    dec.observed.plot(ax = axs[0], ylabel = "Observed")
    axs[0].set_xticklabels([])
    axs[0].set_xlabel(None)
    dec.trend.plot(ax = axs[1], ylabel = "Trend")
    axs[1].set_xticklabels([])
    axs[1].set_xlabel(None)
    dec.seasonal.plot(ax = axs[2], ylabel = "Season")
    axs[2].set_xticklabels([])
    axs[2].set_xlabel(None)
    dec.resid.plot(ax = axs[3], ylabel = "Resid")

In [ ]:
# Decomposition
decs = {}
for c in conditions:
    decs[c] = {}
    for f in fields:
        decs[c][f] = tsa.STL(ts[c][f], robust = True).fit()

In [ ]:
# Visualization
for c in conditions:
    fig, axs = plt.subplots(len(fields), 4, figsize=(24, 24), constrained_layout = True)
    a = axs.T.flatten()
    for i,f in enumerate(fields):
        plot_decompose(decs[c][f], f, a[i*4:(i+1)*4])
    fig.suptitle(c,fontsize = 16)
    fig.savefig("plots/STL/naive_"+c)

In [ ]:
# Separate visualization
for c in conditions:
    for f in fields:
        fig , axs = plt.subplots(4,1, figsize=(10, 7), constrained_layout = True)
        plot_decompose(decs[c][f], f, axs)
        fig.suptitle(c,fontsize = 12)
        fig.savefig("plots/STL/naive/"+c+"-"+f)

We save trends, seasonality, and residuals in different dataframes

In [ ]:
ptrends = pd.DataFrame().reindex_like(ts)
pseasons = pd.DataFrame().reindex_like(ts)
premains = pd.DataFrame().reindex_like(ts)
for c in conditions:
    for f in fields:
        ptrends.loc[:,(c,f)] = decs[c][f].trend
        pseasons.loc[:,(c,f)] = decs[c][f].seasonal
        premains.loc[:,(c,f)] = decs[c][f].resid

## 3.2. FFT Seasonalities Detection

In [ ]:
freqs = {}
for c in conditions:
    fig, axs = plt.subplots(4,5, figsize = (16,8), constrained_layout = True)
    freqs[c] = {}
    for i, ax in enumerate(axs.flatten()):
        f, p = sp.signal.periodogram(ts[c][fields[i]]-ptrends[c][fields[i]], 12.0, scaling='spectrum')
        freqs[c][fields[i]] = (f,p)
        ax.plot(f,p)
        ax.set_title(fields[i])
        ax.set_yticklabels([])
        ax.set_xlabel("Frequency (1/year)")
    fig.suptitle("Power Spectrum - "+c)
    fig.savefig("plots/STL/spectrum_"+c)

In [ ]:
for c in conditions:
    for fi in fields:
        f,p = freqs[c][fi]
        ordamp10perc = np.flip(np.argsort(p))[:int(len(p)*.01)]
        f = pd.unique(f[ordamp10perc].round().astype(int))
        freqs[c][fi] = [i for i in f if i > 0]
        # Yearly seasonality seems to be always needed
        if 1 not in freqs[c][fi]:
            freqs[c][fi].append(1)

## 3.3. Multi-Seasonal Decomposition

### 3.3.1. Decomposition

In [ ]:
# Preparing periods
periods = {}
for c in conditions:
    periods[c] = {}
    for f in fields:
        frs = freqs[c][f]
        # Frequency of 5 cannot be turned in an integer period
        periods[c][f] = np.array([ int(12/fr) for fr in frs if fr != 5 ])
        periods[c][f].sort()
        

In [ ]:
trends = pd.DataFrame().reindex_like(ts)
seasons = { c:{f:[] for f in fields} for c in conditions}
remains = pd.DataFrame().reindex_like(ts)

for c in conditions:
    for f in fields:
        for p in periods[c][f]:
            # Remove previous seasonalities
            to_decompose = ts[c][f]
            for s in seasons[c][f]:
                to_decompose = to_decompose-s
            # Decompose and save new seasonality
            decomposed = tsa.STL(to_decompose, period=p, robust=True).fit()
            seasons[c][f].append(decomposed.seasonal)
            # Save trend and residuals
            if p == periods[c][f][-1]:
                trends.loc[:,(c,f)] = decomposed.trend
                remains.loc[:,(c,f)] = decomposed.resid

### 3.3.2. Plotting

In [ ]:
# Plotting
for c in conditions:
    for f in fields:
        pers = periods[c][f]
        fig = plt.figure(figsize = (10, 10), constrained_layout = True)
        gs = mpl.gridspec.GridSpec(3+len(pers),2, figure=fig)
        fig.suptitle(c+" - "+f, fontsize = 12)
        # Observed
        oax = fig.add_subplot(gs[0,:])
        ts[c][f].plot(ax = oax, ylabel = "Observed")
        oax.set_xticklabels([])
        oax.set_xlabel(None)
        # Trend
        tax = fig.add_subplot(gs[1,:])
        trends[c][f].plot(ax = tax, ylabel = "Trend")
        tax.set_xticklabels([])
        tax.set_xlabel(None)
        # Residuals
        rax = fig.add_subplot(gs[2,:])
        remains[c][f].plot(ax = rax, ylabel = "Resid")
        # Seasonalities
        for i, p in enumerate(pers):
            sax = fig.add_subplot(gs[3+i,0])
            seasons[c][f][i].plot(ax = sax, ylabel = "Season"+str(p))
            saax = fig.add_subplot(gs[3+i,1])
            tsp.month_plot(seasons[c][f][i], ax = saax)
            if i != len(pers)-1:
                sax.set_xticklabels([])
                sax.set_xlabel(None)
                saax.set_xticklabels([])
                saax.set_xlabel(None)
        fig.savefig("plots/STL/multi/"+c+"-"+f)

### 3.3.3. Residuals

In [ ]:
for c in conditions:
    fig, axs = plt.subplots(5,4, figsize = (16,8), constrained_layout = True)
    for i,ax in enumerate(axs.flatten()):
        tsp.plot_acf(remains[c][fields[i]], ax, title = fields[i])
    fig.suptitle("Autocorrelations - "+c)
    fig.savefig("plots/STL/multi/resid_autocorr_"+c)

In [ ]:
res_means = {}
for c in conditions:
    res_means[c] = {}
    for f in fields:
        res_means[c][f] = remains[c][f].mean()
pd.DataFrame(res_means)

In [ ]:
for c in conditions:
    fig, axs = plt.subplots(5,4, figsize = (16,8), constrained_layout = True)
    for i, ax in enumerate(axs.flatten()):
        ax.hist(remains[c][fields[i]], bins=20, log = True)
        ax.set_title(fields[i]+" ~ "+str(np.round(res_means[c][fields[i]],2)))
    fig.suptitle("Distribution - "+c)
    fig.savefig("plots/STL/multi/resid_dist_"+c)

## 3.4. Trend Analysis

In [ ]:
ylims = [(0,3), (0, 75), (0, 75)]
for k,c in enumerate(conditions):
    fig, axs = plt.subplots(5,4, figsize = (18, 12), constrained_layout = True)
    for i, ax in enumerate(axs.flatten()):
        ax.plot(trends[c][fields[i]], "r" if i == 0 else "b")
        ax.set_title(fields[i])
        ax.set_ylim(ylims[k])
    fig.suptitle("Citations % - "+c, fontsize = 16)
    fig.savefig("plots/STL/multi_trends_"+c)

In [ ]:
def corrs(c, df):
    #cmap = sns.diverging_palette(250, 15, s=75, l=40, n=9, center="light", as_cmap=True)
    mat = trends[c].loc[:,trends[c].columns!="overall"].corr()
    #mask = np.triu(np.ones_like(mat, dtype=bool))
    #fig, ax = plt.subplots(figsize=(15, 15))
    #sns.heatmap(mat, mask=mask, cmap=cmap, square=True, annot=True, fmt=".1f", ax=ax)
    #fig.suptitle("Trend Correlations - "+c, fontsize = 16, x = .45, y=1.)
    #plt.show()
    clus = sns.clustermap(mat, annot=True, figsize=(14, 14))
    clus.fig.suptitle("Trend Clustermap - "+c, fontsize = 16, y = 1.)
    return clus

### 3.2.1. 'Comp' Condition

In [ ]:
comp_clus = corrs("Comp", trends)
plt.savefig("plots/STL/multi/clust_Comp")

In [ ]:
c1 = ['history','economics','political science','chemistry','art','psychology','sociology','philosophy','medicine','biology']
c2 = ['materials science','computer science','physics']
c3 = ['business','geography','engineering','environmental science','mathematics','geology']
clusters = [c1,c2,c3]
fig, axs = plt.subplots(3, figsize = (10,10), constrained_layout=True)
fig.suptitle("Clusters - Comp", fontsize = 16)
for i, ax in enumerate(axs.flatten()[:3]):
    trends["Comp"][clusters[i]].plot(ylim=(0,3), ax = ax, xlabel = "")
    trends["Comp"]["overall"].plot(label="overall", style=".", ax = ax, xlabel="")
    ax.legend(loc = (1.01, .0))
fig.savefig("plots/STL/multi/clusts_Comp")

### 3.2.2. 'CS' Condition

In [ ]:
comp_clus = corrs("CS", trends)
plt.savefig("plots/STL/multi/clust_CS")

In [ ]:
c1 = ['philosophy','business','history','economics','sociology','political science','physics','geography']
c2 = ['engineering','materials science','computer science', 'mathematics','art']
c3 = ['medicine','psychology','biology','chemistry','geology','environmental science']
clusters = [c1,c2,c3]
fig, axs = plt.subplots(3, figsize = (10,10),constrained_layout = True)
fig.suptitle("Clusters - CS", fontsize = 16)
for i, ax in enumerate(axs.flatten()[:3]):
    trends["CS"][clusters[i]].plot(ylim=(0,75), ax = ax, xlabel = "")
    trends["CS"]["overall"].plot(label="overall", style=".", ax = ax, xlabel="")
    ax.legend(loc = (1.01, .0))
fig.savefig("plots/STL/multi/clusts_CS")

# 4. Prophet - Modelling

## 4.1. Models

In [ ]:
prophets = {}
for c in conditions:
    prophets[c] = {}
    for f in fields:
        tmp = pd.DataFrame(ts[c][f]).reset_index()
        tmp.columns = ["ds","y"]
        mod = pr.Prophet(daily_seasonality = False, weekly_seasonality = False, changepoint_range=.9)
        #mod.add_seasonality(name="quarterly", period=365.25/4, fourier_order=10)
        mod.fit(tmp)
        prophets[c][f] = mod

In [ ]:
forecasts = {}
for c in conditions:
    forecasts[c] = {}
    for f in fields:
        mod = prophets[c][f]
        fut = mod.make_future_dataframe(periods=12*10, freq = "MS")
        forecasts[c][f] = mod.predict(fut)

In [ ]:
for c in conditions:
    fig, axs = plt.subplots(5,4, figsize = (24,18), constrained_layout = True)
    for i,ax in enumerate(axs.flatten()):
        mod = prophets[c][fields[i]]
        forecast = forecasts[c][fields[i]]
        forecast = forecast[:len(forecast)-120]
        ax.plot(ts[c][fields[i]])
        #mod.plot(forecast, ax=ax, plot_cap=False, xlabel=None, ylabel="", xlabel="")
        ax.plot(forecast["ds"], forecast["yhat"])
        ax.set_title(fields[i])
    fig.suptitle(c, y=1.01)
    fig.savefig("plots/Prophet/in-out_"+c)

## 4.2. Seasonality Plots

In [ ]:
# From https://github.com/facebook/prophet/blob/main/python/prophet/plot.py
from matplotlib.dates import num2date

def set_y_as_percent(ax):
    yticks = 100 * ax.get_yticks()
    yticklabels = ['{0:.4g}%'.format(y) for y in yticks]
    ax.set_yticks(ax.get_yticks().tolist())
    ax.set_yticklabels(yticklabels)
    return ax

def plot_seasonality(m, name, ax=None, figsize=(10, 6)):
    """Plot a custom seasonal component.
    Parameters
    ----------
    m: Prophet model.
    name: Seasonality name, like 'daily', 'weekly'.
    ax: Optional matplotlib Axes to plot on. One will be created if
        this is not provided.
    uncertainty: Optional boolean to plot uncertainty intervals, which will
        only be done if m.uncertainty_samples > 0.
    figsize: Optional tuple width, height in inches.
    Returns
    -------
    a list of matplotlib artists
    """
    artists = []
    if not ax:
        fig = plt.figure(facecolor='w', figsize=figsize)
        ax = fig.add_subplot(111)
    # Compute seasonality from Jan 1 through a single period.
    start = pd.to_datetime('2017-01-01 0000')
    period = m.seasonalities[name]['period']
    end = start + pd.Timedelta(days=period)
    plot_points = 200
    days = pd.to_datetime(np.linspace(start.value, end.value, plot_points))
    df_y = pr.plot.seasonality_plot_df(m, days)
    # NOTE: changes here
    ax.grid(True, which='major', c='gray', ls='-', lw=1, alpha=0.2)
    n_ticks = round(12*period/365.25)
    xticks = pd.to_datetime(np.linspace(start.value, end.value, n_ticks)).to_pydatetime()
    seas = m.predict_seasonal_components(df_y)[name]
    seas = np.array_split(seas, n_ticks)
    seas = [i.mean() for i in seas]
    artists += ax.plot(xticks, seas, ls='-', c='#0072B2')
    # NOTE: changes here (uncertainty missing)
    ax.set_xticks(xticks)
    # NOTE: changes here
    fmt = mpl.ticker.FuncFormatter(
        lambda x, pos=None: '{dt:%b}'.format(dt=num2date(x)))
    ax.set_xlabel('Month')
    ax.xaxis.set_major_formatter(fmt)
    ax.set_ylabel(name)
    if m.seasonalities[name]['mode'] == 'multiplicative':
        ax = set_y_as_percent(ax)
    return artists

In [ ]:
# Plot seasonality
season_name = "yearly"
for c in conditions[:2]:
    fig, axs = plt.subplots(5,4, figsize = (18,12), constrained_layout = True)
    for i,ax in enumerate(axs.flatten()):
        mod = prophets[c][fields[i]]
        pr.plot.plot_yearly(mod, ax = ax)
        #plot_seasonality(mod, name=season_name, ax = ax)
        fmt = mpl.ticker.FuncFormatter(lambda x, pos=None: '{dt:%b}'.format(dt=num2date(x)))
        ax.xaxis.set_major_formatter(fmt)
        ax.set_xlabel("")
        ax.set_title(fields[i])
    fig.suptitle(c)
    fig.savefig("plots/Prophet/seasons_"+c)

## 4.3. Trends

In [ ]:
for c in conditions[:2]:
    fig, axs = plt.subplots(5,4, figsize = (18,12), constrained_layout = True)
    for i,ax in enumerate(axs.flatten()):
        mod = prophets[c][fields[i]]
        forecast = forecasts[c][fields[i]]
        forecast = forecast[:len(forecast)-120]
        pr.plot.plot_forecast_component(mod, forecast, "trend", ax)
        ax.plot(trends[c][fields[i]], 'orange')
        pr.plot.add_changepoints_to_plot(ax, mod, forecast, trend = False)
        ax.set_title(fields[i])
        ax.set_ylabel("")
        ax.set_xlabel("")

    fig.suptitle(c)
    fig.savefig("plots/Prophet/trends_"+c)

## 4.4. Forecasts

In [ ]:
# From prophet.plot.py
def plot(
    m, fcst, ax=None, uncertainty=True, plot_cap=True, xlabel='ds', ylabel='y',
    figsize=(10, 6), include_legend=False
):
    """Plot the Prophet forecast.
    Parameters
    ----------
    m: Prophet model.
    fcst: pd.DataFrame output of m.predict.
    ax: Optional matplotlib axes on which to plot.
    uncertainty: Optional boolean to plot uncertainty intervals, which will
        only be done if m.uncertainty_samples > 0.
    plot_cap: Optional boolean indicating if the capacity should be shown
        in the figure, if available.
    xlabel: Optional label name on X-axis
    ylabel: Optional label name on Y-axis
    figsize: Optional tuple width, height in inches.
    include_legend: Optional boolean to add legend to the plot.
    Returns
    -------
    A matplotlib figure.
    """
    if ax is None:
        fig = plt.figure(facecolor='w', figsize=figsize)
        ax = fig.add_subplot(111)
    else:
        fig = ax.get_figure()
    fcst_t = fcst['ds'].dt.to_pydatetime()
    ax.plot(fcst_t, fcst['yhat'], ls='-', c='#0072B2', label='Forecast')
    if 'cap' in fcst and plot_cap:
        ax.plot(fcst_t, fcst['cap'], ls='--', c='k', label='Maximum capacity')
    if m.logistic_floor and 'floor' in fcst and plot_cap:
        ax.plot(fcst_t, fcst['floor'], ls='--', c='k', label='Minimum capacity')
    if uncertainty and m.uncertainty_samples:
        ax.fill_between(fcst_t, fcst['yhat_lower'], fcst['yhat_upper'],
                        color='#0072B2', alpha=0.2, label='Uncertainty interval')
    # Specify formatting to workaround matplotlib issue #12925
    locator = mpl.dates.AutoDateLocator(interval_multiples=False)
    formatter = mpl.dates.AutoDateFormatter(locator)
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(formatter)
    ax.grid(True, which='major', c='gray', ls='-', lw=1, alpha=0.2)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    if include_legend:
        ax.legend()
    return fig

In [ ]:
for c in conditions:
    fig, axs = plt.subplots(5,4, figsize = (18,12), constrained_layout = True)
    for i,ax in enumerate(axs.flatten()):
        mod = prophets[c][fields[i]]
        forecast = forecasts[c][fields[i]]
        forecast = forecast[len(ts[c][fields[i]]):len(forecast)-60]
        ax.plot(ts[c][fields[i]][12*35:])
        plot(mod, forecast, ax=ax, plot_cap=False, xlabel=None, ylabel="%",)
        ax.set_title(fields[i])
    fig.suptitle(c)
    fig.savefig("plots/Prophet/forecasts_"+c)

## 4.5. Evaluation

In [ ]:
pro_cvs = {}
for c in conditions:
    pro_cvs[c] = {}
    for f in fields:
        res = pr.diagnostics.cross_validation(prophets[c][f], initial = str(365.25*30)+' days', horizon = str(365.25*5)+' days', parallel = 'threads')
        pro_cvs[c][f] = pr.diagnostics.performance_metrics(res, monthly=True)

In [ ]:
for c in conditions:
    fig, axs = plt.subplots(5,4, figsize = (18,12), constrained_layout = True)
    for i, ax in enumerate(axs.flatten()):
        cvres = pro_cvs[c][fields[i]]
#        pr.plot.plot_cross_validation_metric(cvres, "mape")
        ax.plot([ t for t in cvres["horizon"]],cvres['mape']*100)
        ax.set_title(fields[i])
        if i > 15:
            ax.set_xlabel("Horizon (months)")
        if i%4 == 0:
            ax.set_ylabel("% Error")
    fig.suptitle(c)
    fig.savefig("plots/Prophet/evals_"+c)